In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#### Import the data cleaned in another notebook. Set the NDB Number to the index.

In [2]:
nutrients_matrix = pd.read_csv(r"../../data/Nutrients_Branded_Foods_2018/Matrix_Nutrients_Branded_Foods_2018.csv.gz")
print(nutrients_matrix.shape)
nutrients_matrix.set_index("NDB_No", inplace = True)
nutrients_matrix.head()

(237837, 95)


,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,Energy,"Fatty acids, total saturated","Fatty acids, total trans","Fiber, total dietary","Iron, Fe",Protein,"Sodium, Na",...,18:2 undifferentiated,Glucose (dextrose),Starch,Lutein + zeaxanthin,"Fluoride, F","18:2 n-6 c,c","Choline, from phosphotidyl choline","18:3 n-3 c,c,c (ALA)",Epigallocatechin-3-gallate,Vitamin D3 (cholecalciferol)
NDB_No,,,,,,,,,,,,,,,,,,,,,
45001524,50.0,35.00,25.0,200.0,3.75,0.0,0.0,0.0,2.5,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001528,0.0,43.24,0.0,162.0,0.00,0.0,0.0,0.0,0.0,703.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001529,0.0,41.18,0.0,176.0,0.00,0.0,0.0,0.0,0.0,676.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001530,0.0,34.29,0.0,143.0,0.00,0.0,0.0,0.0,0.0,971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45001531,0.0,45.95,0.0,189.0,0.00,0.0,0.0,0.0,0.0,757.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Create two scaled version of the data: MinMax and Standard

In [3]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(nutrients_matrix)
nutrients_min_max_scaled = min_max_scaler.transform(nutrients_matrix)

In [4]:
standard_scaler = StandardScaler()
standard_scaler.fit(nutrients_matrix)
nutrients_standard_scaled = standard_scaler.transform(nutrients_matrix)

#### Do PCA on all datasets

In [11]:
pca = PCA(svd_solver = 'full')
pca.fit(nutrients_matrix)
nutrients_pca = pca.transform(nutrients_matrix)
#nutrients_pca = pd.DataFrame(nutrients_pca, columns = nutrients_matrix.columns)
nutrients_pca.components_

AttributeError: 'numpy.ndarray' object has no attribute 'components_'

In [14]:
pca.explained_variance_ratio_ 

array([5.12484379e-01, 3.60957477e-01, 1.24602479e-01, 1.66455605e-03,
       1.63530302e-04, 8.90034668e-05, 1.82735740e-05, 8.57533531e-06,
       4.99858147e-06, 1.87640449e-06, 1.30641229e-06, 1.28870136e-06,
       7.38253315e-07, 6.39931212e-07, 2.77928104e-07, 1.48366617e-07,
       1.06382416e-07, 8.35590498e-08, 4.57524364e-08, 3.94769996e-08,
       3.89999946e-08, 2.36772740e-08, 2.27334422e-08, 2.26581305e-08,
       2.12561230e-08, 1.92681048e-08, 1.04340452e-08, 3.89835220e-09,
       2.61875551e-09, 1.81994092e-09, 1.51455414e-09, 1.40876562e-09,
       1.07103843e-09, 8.86918863e-10, 8.52900530e-10, 6.75752997e-10,
       6.18463483e-10, 3.16920381e-10, 3.05357086e-10, 2.58307766e-10,
       2.30744623e-10, 1.73944795e-10, 1.50446247e-10, 1.05663837e-10,
       8.57200682e-11, 7.10119719e-11, 3.90746511e-11, 2.92399943e-11,
       2.15010169e-11, 1.45372764e-11, 9.41803652e-12, 4.41670362e-12,
       3.82850549e-12, 3.55194764e-12, 1.35302149e-12, 9.29412207e-13,
      

In [ ]:
nutrient_df.to_csv(r"../../data/Nutrients_Branded_Foods_2018/Matrix_Nutrients_Branded_Foods_2018.csv.gz", 
                   index = False, compression = "gzip")

In [38]:
pca = PCA()
pca.fit(nutrients_min_max_scaled)
data_pca = pca.transform(nutrients_min_max_scaled)

In [38]:
pca = PCA()
pca.fit(nutrients_min_max_scaled)
data_pca = pca.transform(nutrients_min_max_scaled)

#### Loop through the three datasets (original, minmax scaled and standard scaled) for different variance thresholds
We would like to create a table for comparing the best strategy moving forward

In [6]:
data = [nutrients_matrix, nutrients_min_max_scaled, nutrients_standard_scaled]

In [13]:
#create a list of the three datasets for looping
data = [nutrients_matrix, nutrients_min_max_scaled, nutrients_standard_scaled]
#list which will store our results for each dataset
results = []
for j in range(len(data)):
    variance_columns = {}
    #get variance explanation thresholds from 0.75 to 0.95 and record number of columns needed
    #reason (from sklearn docs):
    #If 0 < n_components < 1 and svd_solver == 'full', select the number of components such that the amount of 
    #variance that needs to be explained is greater than the percentage specified by n_components.
    for threshold in np.linspace(0.75, 1, num=5, endpoint=False):
        #set pca using the variance explanation requirement
        pca = PCA(n_components = threshold, svd_solver = 'full')
        pca.fit(data[j])
        pca_result = pca.transform(data[j])
        variance_columns[threshold] = pca_result.shape[1]
    results.append(variance_columns)

In [21]:
g = pd.DataFrame(results)
g = g.T
g.reset_index(inplace = True)
g.rename(columns = {"index": "% Variance Explained", 0: "Original DF", 1: "MinMaxScaled Df", 2: "StandardScaled Df"}, inplace = True)
g

,% Variance Explained,Original DF,MinMaxScaled Df,StandardScaled Df
0,0.75,2,2,38
1,0.80,2,3,42
2,0.85,2,3,47
3,0.90,3,4,52
4,0.95,3,7,59
